In [2]:
from pyspark.sql import SparkSession
import numpy as np

In [3]:
spark = SparkSession.builder.appName("xG4").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 10:08:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [5]:
import pp_events as pp

In [6]:
events_shot = pp.preprocessing(events,spark)

Data loaded
Spatial data calculated
Preferred foot calculated
Goal column created


Number of players inside the area calculated
Dummies created
Boolean data converted to integer


In [7]:
events_shot.columns

['id',
 'player_id',
 'shot_location_x',
 'shot_location_y',
 'distance_to_goal',
 'shot_angle',
 'preferred_foot_shot',
 'other_pp',
 'from_fk',
 'from_ti',
 'from_corner',
 'from_counter',
 'from_gk',
 'from_keeper',
 'from_ko',
 'header',
 'corner_type',
 'fk_type',
 'pk_type',
 'half_volley_technique',
 'volley_technique',
 'lob_technique',
 'overhead_technique',
 'backheel_technique',
 'diving_h_technique',
 'under_pressure',
 'shot_aerial_won',
 'shot_first_time',
 'shot_one_on_one',
 'shot_open_goal',
 'shot_follows_dribble',
 'players_inside_area',
 'shot_statsbomb_xg',
 'shot_outcome',
 'goal']

In [8]:
events_shot.show()

24/12/24 10:09:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/24 10:10:05 WARN TaskSetManager: Stage 5 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|other_pp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_technique|diving_h_technique|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|players_inside_area|shot_statsbomb_xg|shot_outcome|goal|
+-----------------

In [9]:
# since the dataframe is final, we save it into a variable in memory
events_shot = events_shot.cache()

In [11]:
events_shot.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_id: double (nullable = false)
 |-- shot_location_x: float (nullable = false)
 |-- shot_location_y: float (nullable = false)
 |-- distance_to_goal: double (nullable = false)
 |-- shot_angle: float (nullable = false)
 |-- preferred_foot_shot: integer (nullable = true)
 |-- other_pp: integer (nullable = false)
 |-- from_fk: integer (nullable = false)
 |-- from_ti: integer (nullable = false)
 |-- from_corner: integer (nullable = false)
 |-- from_counter: integer (nullable = false)
 |-- from_gk: integer (nullable = false)
 |-- from_keeper: integer (nullable = false)
 |-- from_ko: integer (nullable = false)
 |-- header: integer (nullable = false)
 |-- corner_type: integer (nullable = false)
 |-- fk_type: integer (nullable = false)
 |-- pk_type: integer (nullable = false)
 |-- half_volley_technique: integer (nullable = false)
 |-- volley_technique: integer (nullable = false)
 |-- lob_technique: integer (nullable = false)
 |-- overhead_techniq

In [12]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [13]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline


# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)
# Create the pipeline
pipeline = Pipeline(stages=[log_reg])

In [15]:
# Still rerunning all, fix that
xg_model = pipeline.fit(train_data)

24/12/24 10:13:04 WARN TaskSetManager: Stage 28 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.
24/12/24 10:13:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [16]:
# Make predictions
predictions = xg_model.transform(test_data)

# Evaluate using BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.820818610129564


In [17]:
from pyspark.sql.functions import col, udf, format_number, abs
from pyspark.sql.types import DoubleType

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", abs(col('difference')))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.04334119       |0.0409784791    |0.0023627109 |0.0023627109  |
|0.05638644       |0.0583915642    |-0.0020051242|0.0020051242  |
|0.044450935      |0.0381771463    |0.0062737887 |0.0062737887  |
|0.03511139       |0.0459965398    |-0.0108851498|0.0108851498  |
|0.049519155      |0.0715260485    |-0.0220068935|0.0220068935  |
|0.022752915      |0.0312808028    |-0.0085278878|0.0085278878  |
|0.0752324        |0.1003859203    |-0.0251535203|0.0251535203  |
|0.11348543       |0.1781116410    |-0.0646262110|0.064626211   |
|0.026074365      |0.0166449473    |0.0094294177 |0.0094294177  |
|0.026830109      |0.0145942530    |0.0122358560 |0.012235856   |
|0.086427316      |0.0831530605    |0.0032742555 |0.0032742555  |
|0.05394276       |0.1055020245    |-0.0515592645|0.0515592645  |
|0.0138394

In [18]:
# average of the difference
from pyspark.sql.functions import avg

avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
print(f"Average difference: {avg_diff}")

abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.0029665021397145873


Abs average difference: 0.035255262138028036


In [19]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

from pyspark.sql.functions import round

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

predictions_with_goal_prob.select("goal", "goal_probability_rounded", "shot_statsbomb_xg_rounded").show()

+----+------------------------+-------------------------+
|goal|goal_probability_rounded|shot_statsbomb_xg_rounded|
+----+------------------------+-------------------------+
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|        

In [20]:
from pyspark.sql.functions import col, when

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0)
).withColumn(
    "statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [22]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 90.71%
StatsBomb xG Accuracy: 90.90%


In [23]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

other_pp: 0.23504317481796677
from_fk: -0.014655272218302222
from_ti: 0.026822398493000006
from_corner: -0.2970947477534408
from_counter: 0.3024488773252327
from_gk: 0.042823770556808284
from_keeper: 0.07951789377610598
from_ko: -0.04485029107382409
header: -0.9453129787692491
corner_type: 2.4605984497564206
fk_type: 1.6585385070535832
pk_type: 1.736649971325377
half_volley_technique: -0.46142540497427303
volley_technique: -0.4973456185072562
lob_technique: 0.920062407765215
overhead_technique: -1.2066641562576674
backheel_technique: -1.2530705961344488
diving_h_technique: 0.623384495779672
distance_to_goal: -0.10062640890219708
shot_angle: 0.027680445046920577
preferred_foot_shot: 0.2935566964779566
under_pressure: -0.08681428590577206
shot_aerial_won: -0.4999203102655947
shot_first_time: 0.0594210045906572
shot_one_on_one: 0.24682148842066517
shot_open_goal: 0.8283854790363725
shot_follows_dribble: -0.3678427674272859
players_inside_area: -0.29950999453588895


In [24]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [25]:
xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 98.54%
